In [1]:
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import numpy as np
import urllib.request
from urllib.request import urlopen
from bs4 import BeautifulSoup
import csv
import plotly.express as px
import plotly

pd.options.mode.chained_assignment = None  # default='warn'


In [2]:
url = 'https://www.foxsports.com/nba/washington-wizards-team-game-log'
page = urlopen(url)
soup = BeautifulSoup(page,'html.parser')
table = soup.find('table')

In [3]:
output_rows = []
for table_row in table.findAll('tr'):
    columns = table_row.findAll('td')
    output_row = []
    for column in columns:
        output_row.append(column.text)
    output_rows.append(output_row)

output_rows = output_rows[1:]


In [4]:
output_rows

[[' 10/20\n',
  ' @ TOR\n',
  ' W 98-83\n',
  '36',
  '81',
  '44.400',
  '21',
  '26',
  '80.800',
  '5',
  '27',
  '18.500',
  '98'],
 [' 10/22\n',
  ' vs IND\n',
  ' W 135-134\n',
  '49',
  '94',
  '52.100',
  '18',
  '19',
  '94.700',
  '19',
  '38',
  '50.000',
  '135'],
 [' 10/25\n',
  ' @ BKN\n',
  ' L 90-104\n',
  '33',
  '95',
  '34.700',
  '15',
  '20',
  '75.000',
  '9',
  '39',
  '23.100',
  '90'],
 [' 10/27\n',
  ' @ BOS\n',
  ' W 116-107\n',
  '43',
  '84',
  '51.200',
  '21',
  '29',
  '72.400',
  '9',
  '25',
  '36.000',
  '116'],
 [' 10/28\n',
  ' vs ATL\n',
  ' W 122-111\n',
  '46',
  '99',
  '46.500',
  '17',
  '21',
  '81.000',
  '13',
  '34',
  '38.200',
  '122'],
 [' 10/30\n',
  ' vs BOS\n',
  ' W 115-112\n',
  '38',
  '104',
  '36.500',
  '29',
  '36',
  '80.600',
  '10',
  '36',
  '27.800',
  '115'],
 [' 11/01\n',
  ' @ ATL\n',
  ' L 111-118\n',
  '41',
  '86',
  '47.700',
  '16',
  '16',
  '100.000',
  '13',
  '39',
  '33.300',
  '111'],
 [' 11/03\n',
  ' vs TO

In [5]:
# date_index = []
# for game in range(0,len(output_rows)):
#     date_index.append(output_rows[game][0])

# date_index

In [6]:
column_values = ['Date','Opponent','Score','FGM','FGA','FG%','FTM','FTA','FT%','3FGM','3FGA','3FG%','PTS']

# Create a dataframe
df = pd.DataFrame(data = output_rows,
                  columns = column_values)
df

,Date,Opponent,Score,FGM,FGA,FG%,FTM,FTA,FT%,3FGM,3FGA,3FG%,PTS
0,10/20\n,@ TOR\n,W 98-83\n,36,81,44.400,21,26,80.800,5,27,18.500,98
1,10/22\n,vs IND\n,W 135-134\n,49,94,52.100,18,19,94.700,19,38,50.000,135
2,10/25\n,@ BKN\n,L 90-104\n,33,95,34.700,15,20,75.000,9,39,23.100,90
3,10/27\n,@ BOS\n,W 116-107\n,43,84,51.200,21,29,72.400,9,25,36.000,116
4,10/28\n,vs ATL\n,W 122-111\n,46,99,46.500,17,21,81.000,13,34,38.200,122
...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,04/03\n,@ BOS\n,,,,,,,,,,,
78,04/05\n,@ MIN\n,,,,,,,,,,,
79,04/06\n,@ ATL\n,,,,,,,,,,,
80,04/08\n,vs NYK\n,,,,,,,,,,,


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82 entries, 0 to 81
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Date      82 non-null     object
 1   Opponent  82 non-null     object
 2   Score     82 non-null     object
 3   FGM       82 non-null     object
 4   FGA       82 non-null     object
 5   FG%       82 non-null     object
 6   FTM       82 non-null     object
 7   FTA       82 non-null     object
 8   FT%       82 non-null     object
 9   3FGM      82 non-null     object
 10  3FGA      82 non-null     object
 11  3FG%      82 non-null     object
 12  PTS       82 non-null     object
dtypes: object(13)
memory usage: 8.5+ KB


In [8]:

# Remove /n from Date, Opponent, and Score Columns
df['Date'] = df['Date'].str.replace("\n","")
df['Opponent'] = df['Opponent'].str.replace("\n", " ")

# Remove the whitespace on the left of the Score Column
df['Score'] = df['Score'].str.replace("\n","").str.lstrip()
# Create a Result Column and Update Score Column
df[['Result','Score']] = df['Score'].str.split(' ',expand = True)
# Create Opponents Score Column
df['OpScore'] = df['Score'].str.split('-').str[1]

# Home or Away Column
df['H/A'] = df['Opponent'].str.split(' ').apply(lambda x: 'Away' if (x[1] == '@') else 'Home')
                                               
# Drop Games that haven't been played yet from the data
df = df.dropna(axis = 0, how = 'any')

# Covert the following columns to the int type
int_cols = ['FGM','FGA','FTM','FTA','3FGM','3FGA','PTS','OpScore']
df[int_cols] = df[int_cols].astype(int)
# Convert the following columns to the float type
float_cols = ['FG%','FT%','3FG%']
df[float_cols] = df[float_cols].astype(float)


# Format Opponent Column
df['Opponent'] = df['Opponent'].str.replace('@','')
df['Opponent'] = df['Opponent'].str.replace('vs','')
df

,Date,Opponent,Score,FGM,FGA,FG%,FTM,FTA,FT%,3FGM,3FGA,3FG%,PTS,Result,OpScore,H/A
0,10/20,TOR,98-83,36,81,44.4,21,26,80.8,5,27,18.5,98,W,83,Away
1,10/22,IND,135-134,49,94,52.1,18,19,94.7,19,38,50.0,135,W,134,Home
2,10/25,BKN,90-104,33,95,34.7,15,20,75.0,9,39,23.1,90,L,104,Away
3,10/27,BOS,116-107,43,84,51.2,21,29,72.4,9,25,36.0,116,W,107,Away
4,10/28,ATL,122-111,46,99,46.5,17,21,81.0,13,34,38.2,122,W,111,Home
5,10/30,BOS,115-112,38,104,36.5,29,36,80.6,10,36,27.8,115,W,112,Home
6,11/01,ATL,111-118,41,86,47.7,16,16,100.0,13,39,33.3,111,L,118,Away
7,11/03,TOR,100-109,34,78,43.6,24,30,80.0,8,32,25.0,100,L,109,Home
8,11/05,MEM,115-87,49,84,58.3,7,13,53.8,10,25,40.0,115,W,87,Home
9,11/07,MIL,101-94,41,81,50.6,8,13,61.5,11,27,40.7,101,W,94,Home


In [9]:

df

,Date,Opponent,Score,FGM,FGA,FG%,FTM,FTA,FT%,3FGM,3FGA,3FG%,PTS,Result,OpScore,H/A
0,10/20,TOR,98-83,36,81,44.4,21,26,80.8,5,27,18.5,98,W,83,Away
1,10/22,IND,135-134,49,94,52.1,18,19,94.7,19,38,50.0,135,W,134,Home
2,10/25,BKN,90-104,33,95,34.7,15,20,75.0,9,39,23.1,90,L,104,Away
3,10/27,BOS,116-107,43,84,51.2,21,29,72.4,9,25,36.0,116,W,107,Away
4,10/28,ATL,122-111,46,99,46.5,17,21,81.0,13,34,38.2,122,W,111,Home
5,10/30,BOS,115-112,38,104,36.5,29,36,80.6,10,36,27.8,115,W,112,Home
6,11/01,ATL,111-118,41,86,47.7,16,16,100.0,13,39,33.3,111,L,118,Away
7,11/03,TOR,100-109,34,78,43.6,24,30,80.0,8,32,25.0,100,L,109,Home
8,11/05,MEM,115-87,49,84,58.3,7,13,53.8,10,25,40.0,115,W,87,Home
9,11/07,MIL,101-94,41,81,50.6,8,13,61.5,11,27,40.7,101,W,94,Home


In [42]:
# home_win_loss 
home_wins = len(df[(df['Result']=='W') & (df['H/A'] == 'Home')])
home_losses = len(df[(df['Result']=='L') & (df['H/A'] == 'Home')])
home_wl = str(home_wins) + '-' + str(home_losses)
home_win_percentage = str((home_wins/(home_wins+home_losses)*100)) + '%'

# away_win_loss
road_wins = len(df[(df['Result']=='W') & (df['H/A'] == 'Away')])
road_losses = len(df[(df['Result']=='L') & (df['H/A'] == 'Away')])
road_wl = str(road_wins) + '-' + str(road_losses)

#overall win_loss
wins = len(df[df['Result'] == 'W'])
losses = len(df[df['Result'] == 'L'])
wl = str(wins) + '-' + str(losses)
road_wl, home_wl

('4-2', '6-1')

In [11]:
wins = []
winpercentage = []
def rollingWinPercentage(series):
    result = series
    for element in range(0,len(result)):
        if result[element] == 'W':
            wins.append(1)
        else:
            wins.append(0)
            
        wp = (sum(wins)/len(wins))
        winpercentage.append(round(wp,2))
        
    return winpercentage
            

In [12]:
Win_Percentage = pd.Series(rollingWinPercentage(df['Result']))
Win_Percentage

0     1.00
1     1.00
2     0.67
3     0.75
4     0.80
5     0.83
6     0.71
7     0.62
8     0.67
9     0.70
10    0.73
11    0.75
12    0.77
dtype: float64

In [13]:
# Feature Engineering

# Create a point differential column
df['PointDiff'] = df['PTS'] - df['OpScore']

# Effective Field Goal Percentage
df['EFG%'] = (((df['FGM'] + 0.5*df['3FGM'])/df['FGA'])*100).round(2)

# Free Throw Rate
df['FTR'] = ((df['FTM']/df['FGA'])*100).round(2)

# Win Percentage Column
df['Win Percentage'] = Win_Percentage

df



,Date,Opponent,Score,FGM,FGA,FG%,FTM,FTA,FT%,3FGM,3FGA,3FG%,PTS,Result,OpScore,H/A,PointDiff,EFG%,FTR,Win Percentage
0,10/20,TOR,98-83,36,81,44.4,21,26,80.8,5,27,18.5,98,W,83,Away,15,47.53,25.93,1.00
1,10/22,IND,135-134,49,94,52.1,18,19,94.7,19,38,50.0,135,W,134,Home,1,62.23,19.15,1.00
2,10/25,BKN,90-104,33,95,34.7,15,20,75.0,9,39,23.1,90,L,104,Away,-14,39.47,15.79,0.67
3,10/27,BOS,116-107,43,84,51.2,21,29,72.4,9,25,36.0,116,W,107,Away,9,56.55,25.00,0.75
4,10/28,ATL,122-111,46,99,46.5,17,21,81.0,13,34,38.2,122,W,111,Home,11,53.03,17.17,0.80
5,10/30,BOS,115-112,38,104,36.5,29,36,80.6,10,36,27.8,115,W,112,Home,3,41.35,27.88,0.83
6,11/01,ATL,111-118,41,86,47.7,16,16,100.0,13,39,33.3,111,L,118,Away,-7,55.23,18.60,0.71
7,11/03,TOR,100-109,34,78,43.6,24,30,80.0,8,32,25.0,100,L,109,Home,-9,48.72,30.77,0.62
8,11/05,MEM,115-87,49,84,58.3,7,13,53.8,10,25,40.0,115,W,87,Home,28,64.29,8.33,0.67
9,11/07,MIL,101-94,41,81,50.6,8,13,61.5,11,27,40.7,101,W,94,Home,7,57.41,9.88,0.70


In [28]:
def lineplot(dataframe, xcol, ycol, Graph_title, font_style = 'Georgia'):
    
    fig = px.line(dataframe, x = xcol, y = ycol, title = Graph_title,template = 'simple_white',markers = True)

    # fig = px.line(df, x = 'Date', y = 'PointDiff', title = 'Point Differential',template = 'simple_white')
    fig.update_layout(font_family = font_style, title_x = 0.5, font = dict(size = 18, color = 'Black'))
    
    return fig

In [29]:
lineplot(df, 'Date', 'PointDiff', 'Point Differential')

# Statistics for Dashboard

Effective Field Goal Percentage

3 Point Percentage

Free Throw Percentage

In [16]:
EFGP = df['EFG%'].mean()
Perc_3 = df['3FG%'].mean()
FTP = df['FT%'].mean()


print('Expected Field Goal Percentage: ' + str(round(EFGP,2)) + '%')
print('3PT Field Goal Percentage: ' + str(round(Perc_3,2))+ '%')
print('Free Throw Percentage: ' + str(round(FTP,2))+ '%')


Expected Field Goal Percentage: 52.53%
3PT Field Goal Percentage: 33.72%
Free Throw Percentage: 79.06%


In [17]:
# Proportion of Home Games Pie Chart

# Count of home games
count_home = len(df[df['H/A'] == 'Home'])
# Count List of away games
count_away = len(df[df['H/A'] == 'Away'])
# To a List
value = [count_home, count_away]
name = ['Home', 'Away']


# Pie Chart
fig = px.pie(df,values = value,names = name, title='Proportion of Home Games')
fig.update_layout(font_family = 'Georgia', title_x = 0.5, font = dict(size = 18))
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.update_layout(showlegend=False)



fig.show()

In [18]:
fig = px.line(df, x = 'Date', y = 'EFG%', title = 'Effective Field Goal Percentage',template = 'plotly_white')
fig.update_layout(font_family = 'Georgia', title_x = 0.5, font = dict(size = 18))
fig.show()